In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
fake = pd.read_csv('D:\\6th Sem\\Project- Fake News Detection\\Python Model FN\\Fake.csv',encoding='latin1')
true = pd.read_csv('D:\\6th Sem\\Project- Fake News Detection\\Python Model FN\\True.csv',encoding='latin1')

In [3]:
fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-Dec-17
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-Dec-17
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-Dec-17
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-Dec-17
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-Dec-17


In [5]:
fake['target'] = 'FAKE'
true['target'] = 'REAL'

In [6]:
data = pd.concat([fake, true]).reset_index(drop = True)
data

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",FAKE
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",FAKE
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-Aug-17,REAL
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-Aug-17,REAL
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-Aug-17,REAL
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-Aug-17,REAL


In [7]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,title,text,subject,date,target
0,Pentagon to send about a dozen Guantanamo inma...,WASHINGTON (Reuters) - The Pentagon plans to t...,politicsNews,31-Mar-16,REAL
1,This Gay Coupleâs Trolling Of A Neo-Nazi Ra...,"Racists, homophobes, misogynists and other big...",News,"May 24, 2016",FAKE
2,Caught On Camera: Devastating Moment 2100 Peo...,Some companies definitely prefer increased pro...,News,"February 12, 2016",FAKE
3,Students protest U.S. Attorney General speech ...,WASHINGTON (Reuters) - Students and faculty at...,politicsNews,26-Sep-17,REAL
4,Trump's son-in-law to oversee government revamp,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,27-Mar-17,REAL


In [8]:
data.drop(["date"],axis=1,inplace=True)
data.drop(["title"],axis=1,inplace=True)
data

,text,subject,target
0,WASHINGTON (Reuters) - The Pentagon plans to t...,politicsNews,REAL
1,"Racists, homophobes, misogynists and other big...",News,FAKE
2,Some companies definitely prefer increased pro...,News,FAKE
3,WASHINGTON (Reuters) - Students and faculty at...,politicsNews,REAL
4,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,REAL
...,...,...,...
44893,HOUSE SPEAKER PAUL RYAN HAS SOUR GRAPES ABOUT ...,Government News,FAKE
44894,HOUSTON (Reuters) - A Republican-led congressi...,politicsNews,REAL
44895,Admit it: Ever since news broke that Donald Tr...,News,FAKE
44896,Donald Trump s credibility and power has bec...,News,FAKE


In [9]:
data.isnull().any()

text       False
subject    False
target     False
dtype: bool

In [10]:
data['text'] = data['text'].apply(lambda x: x.lower())
data

,text,subject,target
0,washington (reuters) - the pentagon plans to t...,politicsNews,REAL
1,"racists, homophobes, misogynists and other big...",News,FAKE
2,some companies definitely prefer increased pro...,News,FAKE
3,washington (reuters) - students and faculty at...,politicsNews,REAL
4,washington (reuters) - u.s. president donald t...,politicsNews,REAL
...,...,...,...
44893,house speaker paul ryan has sour grapes about ...,Government News,FAKE
44894,houston (reuters) - a republican-led congressi...,politicsNews,REAL
44895,admit it: ever since news broke that donald tr...,News,FAKE
44896,donald trump s credibility and power has bec...,News,FAKE


In [11]:
import string
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
data['text'] = data['text'].apply(punctuation_removal)
data

,text,subject,target
0,washington reuters the pentagon plans to tran...,politicsNews,REAL
1,racists homophobes misogynists and other bigot...,News,FAKE
2,some companies definitely prefer increased pro...,News,FAKE
3,washington reuters students and faculty at ge...,politicsNews,REAL
4,washington reuters us president donald trumpâ...,politicsNews,REAL
...,...,...,...
44893,house speaker paul ryan has sour grapes about ...,Government News,FAKE
44894,houston reuters a republicanled congressional...,politicsNews,REAL
44895,admit it ever since news broke that donald tru...,News,FAKE
44896,donald trump s credibility and power has bec...,News,FAKE


In [12]:
#stopwords removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head(50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HZT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,subject,target
0,washington reuters pentagon plans transfer doz...,politicsNews,REAL
1,racists homophobes misogynists bigots droves d...,News,FAKE
2,companies definitely prefer increased profit p...,News,FAKE
3,washington reuters students faculty georgetown...,politicsNews,REAL
4,washington reuters us president donald trumpâ...,politicsNews,REAL
5,donald trump brought every racist closet given...,News,FAKE
6,worth noting victims horrible terrorist bombin...,politics,FAKE
7,reuters following people mentioned contenders ...,politicsNews,REAL
8,london reuters british prime minister theresa ...,politicsNews,REAL
9,washington reuters michelle obama travel liber...,politicsNews,REAL


In [17]:
x_train,x_test,y_train,y_test = train_test_split(data['text'], data.target, test_size=0.2)

In [18]:
#tfidf_vectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                    ('lrmodel', LogisticRegression())])

In [20]:
modellr = pipeline.fit(x_train,y_train)

C:\Users\HZT\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
prediction = modellr.predict(x_test)
print("Accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

Accuracy: 99.09%


In [22]:
pred = modellr.predict(x_test)

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

        FAKE       0.99      0.99      0.99      4653
        REAL       0.99      0.99      0.99      4327

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [25]:
print(confusion_matrix(y_test, pred))

[[4606   47]
 [  35 4292]]


In [27]:
print(modellr.predict(["How elite leftists celebrate America s Independence Day The Kennedy clan gathered at their Hyannis Port compound on Cape Cod over the weekend for their annual Fourth of July festivities, and took some time to attack Donald Trump.Robert F. Kennedy Jr. s daughter Kathleen, between known as Kick, posted a photos of a pinata of The Donald from a family party over the weekend. It s yuge party!,  wrote Kick in the caption of the Instagram post, which also showed some of her family members milling about in the background.She later deleted the Instagram post just before 11am on Monday.Via: Daily Mail"]))

['FAKE']


In [28]:
pickle.dump(modellr, open('modellr.pkl','wb'))  
modellr = pickle.load(open('modellr.pkl','rb'))